## Группа DS03-onl

Студент Парфимович Алексей

## Домашнее задание №11
Просрочит ли клиент микро-финансовой организации выплату более чем на 60 дней или нет?  
Построить модель, которая по входным данным предсказывала бы с максимальной точностью ключевой параметр.

In [1]:
import numpy as np
import pandas as pd
import pandas_profiling

from math import nan
from datetime import datetime 

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import GridSearchCV

DATA_CSV_PATH = 'MFOcredit.csv'

### Загрузка и анализ данных

Предварительная загрузка части данных

In [2]:
df_sample = pd.read_csv(DATA_CSV_PATH, sep=';', nrows=3)
df_sample.head()

,id,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,1,03-Jan-2013,12-Jan-2013,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
1,2,03-Jan-2013,17-Jan-2013,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет
2,3,03-Jan-2013,17-Jan-2013,Мужской,25,Пропуск поля,Собственное,Холост,Да,Официальное,Кемеровская область север,Пропуск поля,Нет,Нет


Анализ признаков и их типов

Загрузка требуемых данных: для признаков явно указываем требуемые типы, признак id исключаем из набора

In [3]:
dtypes={
    'date_start': object,
    'date_end': object,
    'gender': object,
    'age': int,
    'auto': object,
    'housing': object,
    'marstatus': object,
    'regclient': object,
    'jobtype': object,
    'region': object,
    'credits': object,
    'children': object,
    'delinq60plus': object
    }

# Cписок признаков
col_list = df_sample.columns.tolist()

# Исключить признак 'id'
cols = col_list[1:]

df_raw = pd.read_csv(DATA_CSV_PATH, usecols = cols, dtype = dtypes, sep=';')

df_raw = df_raw.apply(lambda x: x.astype(str).str.lower() if x.dtype.name=='object' else x)

df_raw.replace('пропуск поля', nan, inplace=True)

df_raw.head()

,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,03-jan-2013,12-jan-2013,мужской,44,нет,собственное,гражданский брак/женат/замужем,нет,официальное,новосибирская область,нет,да,нет
1,03-jan-2013,17-jan-2013,мужской,21,NaN,живут с родителями,холост,нет,официальное,кемеровская область юг,да,нет,нет
2,03-jan-2013,17-jan-2013,мужской,25,NaN,собственное,холост,да,официальное,кемеровская область север,NaN,нет,нет
3,03-jan-2013,17-jan-2013,женский,47,NaN,собственное,гражданский брак/женат/замужем,да,официальное,кемеровская область север,нет,нет,нет
4,03-jan-2013,17-jan-2013,мужской,22,нет,арендуемое,гражданский брак/женат/замужем,нет,официальное,кемеровская область север,да,да,нет


Замена показателей дат начала (date_start) и окончания (date_end) периода займа на интегральный показатель срока займа (loan_term)

In [4]:
df_raw.insert(0,'loan_term', (pd.to_datetime(df_raw['date_end']) - pd.to_datetime(df_raw['date_start'])).dt.days)
df_raw.drop(['date_start','date_end'], inplace=True, axis=1)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35212 entries, 0 to 35211
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   loan_term     35212 non-null  int64 
 1   gender        35212 non-null  object
 2   age           35212 non-null  int32 
 3   auto          21135 non-null  object
 4   housing       27609 non-null  object
 5   marstatus     27648 non-null  object
 6   regclient     35212 non-null  object
 7   jobtype       20755 non-null  object
 8   region        35212 non-null  object
 9   credits       34266 non-null  object
 10  children      34467 non-null  object
 11  delinq60plus  35212 non-null  object
dtypes: int32(1), int64(1), object(10)
memory usage: 3.1+ MB


In [5]:
df_raw['delinq60plus'] = df_raw['delinq60plus'].apply(lambda x: 1 if x == 'да' else 0)
df_raw['delinq60plus'].value_counts()

0    21705
1    13507
Name: delinq60plus, dtype: int64

In [6]:
#df_raw.groupby(cols)['loan_term'].count().sort_values(ascending=False).head(30)

df_duplicateRows = df_raw[df_raw.duplicated()]
df_duplicateRows.shape

(7515, 12)

In [7]:
df_raw = df_raw.drop_duplicates(keep='first')
df_raw.shape

(27697, 12)

Pазвернутый анализ данных c помощью библиотеки __[Pandas Profiling](https://github.com/pandas-profiling/pandas-profiling)__

In [8]:
#%%time
#pandas_profiling.ProfileReport(df_raw)

Выборка строк, содержащих признаки со значением "Пропуск поля"

In [9]:
val = df_raw.apply(lambda row: row.isna().any(), axis=1)
df_raw[val].to_csv('empty_fields.csv')

In [11]:
#categorical_features = ['gender', 'auto', 'housing', 'marstatus', 'regclient', 'jobtype', 'region', 'credits', 'children']
num_columns = [col for col in df_raw.columns if df_raw[col].dtype.name!='object']
cat_columns = [col for col in df_raw.columns if df_raw[col].dtype.name=='object']
#cat_columns.remove('delinq60plus')

df_num = df_raw[num_columns]

#df_dummed = pd.get_dummies(df_raw, columns=categorical_features, drop_first=True, dummy_na=True)
df_dummed = pd.get_dummies(df_raw[cat_columns]) #, dummy_na=True)
df_dummed.head()

,gender_женский,gender_мужской,auto_да,auto_нет,housing_арендуемое,housing_долевая собственность,housing_живут с родителями,housing_муниципальное,housing_собственное,marstatus_вдова/вдовец,...,jobtype_официальное,region_алтайский край,region_кемеровская область север,region_кемеровская область юг,region_красноярский край,region_новосибирская область,credits_да,credits_нет,children_да,children_нет
0,0,1,0,1,0,0,0,0,1,0,...,1,0,0,0,0,1,0,1,1,0
1,0,1,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,1,0,0,1
2,0,1,0,0,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,1,0,...,1,0,1,0,0,0,0,1,0,1
4,0,1,0,1,1,0,0,0,0,0,...,1,0,1,0,0,0,1,0,1,0


In [12]:
data = pd.concat((df_num, df_dummed), axis=1)
data.head()

,loan_term,age,delinq60plus,gender_женский,gender_мужской,auto_да,auto_нет,housing_арендуемое,housing_долевая собственность,housing_живут с родителями,...,jobtype_официальное,region_алтайский край,region_кемеровская область север,region_кемеровская область юг,region_красноярский край,region_новосибирская область,credits_да,credits_нет,children_да,children_нет
0,9,44,0,0,1,0,1,0,0,0,...,1,0,0,0,0,1,0,1,1,0
1,14,21,0,0,1,0,0,0,0,1,...,1,0,0,1,0,0,1,0,0,1
2,14,25,0,0,1,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
3,14,47,0,1,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,0,1
4,14,22,0,0,1,0,1,1,0,0,...,1,0,1,0,0,0,1,0,1,0


In [ ]:
# Матрицу корреляции заменим более компактным и
# достаточно информативным вариантом представления
corrs = data.corr().abs()
np.fill_diagonal(corrs.values, 0)

s = corrs.unstack()
so = s.sort_values(kind="quicksort")
so

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn

sn.heatmap(data.corr(), annot=True, linewidths=1)
plt.show()

In [13]:
y = data['delinq60plus']
X = data.drop(['delinq60plus'], axis=1)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [15]:
# Модель RandomForest
forest = RandomForestClassifier(n_estimators=200, max_depth=10, 
                                random_state=2020, n_jobs=-1)
                                
forest.fit(X_train, y_train);

print('AUC на обучающей выборке: {:.3f}'.
      format(roc_auc_score(y_train, forest.predict_proba(X_train)[:, 1])))
print('AUC на контрольной выборке: {:.3f}'.
      format(roc_auc_score(y_test, forest.predict_proba(X_test)[:, 1])))

AUC на обучающей выборке: 0.755
AUC на контрольной выборке: 0.677
